In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print("GPUs Available:", gpus)

GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("Using GPU:", tf.test.is_gpu_available())

TensorFlow version: 2.10.0
Built with CUDA: True
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Using GPU: True


In [3]:
!nvidia-smi

Fri Oct 31 21:46:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.03                 Driver Version: 577.03         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P0             19W /   80W |    6700MiB /   8188MiB |     12%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import sys
print(sys.executable)


C:\Users\saina\anaconda3\envs\venv_1\python.exe


# `Project Overview`

# `Importing Libraries`

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub


import warnings
warnings.filterwarnings("ignore")

C:\Users\saina\anaconda3\envs\venv_1\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


# `Exploring the Data`

In [6]:
import os
import pandas as pd

base_path = "Emotion Detection from Faces Dataset"

data = []

for folder in ["train", "test"]:
    folder_path = base_path + "/" + folder
    for emotion in os.listdir(folder_path):
        emotion_path = folder_path + "/" + emotion
        count = len(os.listdir(emotion_path))
        data.append({"Set": folder, "Emotion": emotion, "Image Count": count})

df = pd.DataFrame(data)
df

,Set,Emotion,Image Count
0,train,angry,3995
1,train,disgust,436
2,train,fear,4097
3,train,happy,7215
4,train,neutral,4965
5,train,sad,4830
6,train,surprise,3171
7,test,angry,958
8,test,disgust,111
9,test,fear,1024


In [7]:
IMG_HEIGHT = 48
IMG_WIDTH = 48 
IMG_CHANNELS = 3
batch_size = 32

In [8]:
# Preparing the Data

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

In [9]:
train_data = train_datagen.flow_from_directory(
    directory = "Emotion Detection from Faces Dataset/train",
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    class_mode = "categorical",
    subset = "training"
)

val_data = train_datagen.flow_from_directory(
    directory = "Emotion Detection from Faces Dataset/train",
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    class_mode = "categorical",
    subset = "validation"
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [10]:
test_datagen = ImageDataGenerator(
    rescale = 1/255.
)

test_data = test_datagen.flow_from_directory(
    directory = "Emotion Detection from Faces Dataset/test",
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle = False
)

Found 7178 images belonging to 7 classes.


In [11]:
train_data.image_shape

(48, 48, 3)

# `Training a Model`

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation="relu"),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation="relu"),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(units=75, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),

    Dense(25, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),

    Dense(7, activation="softmax")
])

In [ ]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
epochs = 20

In [ ]:
model_history = model.fit(x = train_data,
                          epochs = epochs,
                          steps_per_epoch = int(train_data.samples/batch_size),
                          validation_data = val_data,
                          validation_steps = int(val_data.samples/batch_size),
                          verbose = 1
                         )

we are Gettinng a Accuracy around 0.4469 which is not good

# `Using Transfer Learning`

In [12]:
resnet_50_layer = hub.KerasLayer(
    "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5",
    input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
    trainable=False
)

resnet_50_model = Sequential([
    resnet_50_layer,
    Dense(7, activation='softmax')
])

In [13]:
resnet_50_model.compile(
    loss = 'categorical_crossentropy',
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [14]:
EPOCHS = 10

In [15]:
resnet_50_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23564800  
                                                                 
 dense (Dense)               (None, 7)                 14343     
                                                                 
Total params: 23,579,143
Trainable params: 14,343
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
resnet_50_model_history = resnet_50_model.fit(
    x = train_data,
    epochs = EPOCHS,
    steps_per_epoch = len(train_data),
    validation_data = val_data,
    validation_steps = len(val_data),
    verbose = 1
)